In [1]:
import pandas as pd
import mlflow
from pathlib import Path

# funciones personalizadas
from package_ml.modeling.baseline import train_baselines_with_mlflow

2025-06-20 11:19:20.201 | INFO     | package_ml.config:<module>:11 - PROJ_ROOT path is: /home/fernando/Documentos/azure_ml


In [2]:
df = pd.read_parquet('../data/interim/train_fe.parquet')

In [3]:
df.head()

,User_ID,Gender,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Age_x_Height,...,Pct_FCMT_sq,Pct_FCMT_cu,Duration_log,Heart_Rate_log,Body_Temp_log,Weight_log,Duration_x_Heart_Rate_log,is_temp_high,is_overweight,feno_var
0,14733363,True,68.0,190.0,94.0,29.0,105.0,40.8,231.0,12920.0,...,4771.900970,329637.895921,3.401197,4.663439,3.732896,4.553877,8.021585,True,False,1262.9616
1,14861698,False,20.0,166.0,60.0,14.0,94.0,40.3,66.0,3320.0,...,2209.000000,103823.000000,2.708050,4.553877,3.720862,4.110874,7.183112,True,False,429.6964
2,11179863,True,69.0,179.0,79.0,5.0,88.0,38.7,26.0,12351.0,...,3396.342266,197932.529391,1.791759,4.488636,3.681351,4.382027,6.089045,False,False,150.2240
3,16180408,False,34.0,179.0,71.0,13.0,100.0,40.5,71.0,6086.0,...,2890.507573,155403.632964,2.639057,4.615121,3.725693,4.276666,7.170888,True,False,465.4143
4,17771927,False,27.0,154.0,58.0,10.0,81.0,39.8,35.0,4158.0,...,1761.389568,73923.603610,2.397895,4.406719,3.708682,4.077537,6.698268,True,False,251.4440


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Columns: 141 entries, User_ID to feno_var
dtypes: bool(3), float64(137), int64(1)
memory usage: 15.8 MB


In [5]:
integer_columns = df.select_dtypes(include=['int8', 'int32', 'int64']).columns

In [6]:
integer_columns

Index(['User_ID'], dtype='object')

In [7]:
TARGET = "Calories"
ID_COLUMN = "User_ID"
FEATURES = [col for col in df.columns if col not in [TARGET, ID_COLUMN]]
y = df[TARGET]
X = df[FEATURES]

In [8]:
X.shape

(15000, 139)

## Pasos para los modelos base

El objetivo es entrenar rapidamente varios modelos con sus configuraciones por defecto para tener un rendimiento base a traves de lo siguiente:

- **Modelos a probar**: Ridge, Lasso, HistGradientBoostingRegressor, XGBoost y LightGBM.
- **Validación:** Usaremos una **Validación Cruzada de 10-Folds** para asegurar que nuestras métricas sean estables y no dependan de una única partición de datos.
- **Métricas:** Evaluaremos con RMSE, MAE, etc.
- **Seguimiento de Experimentos:** Utilizaremos **MLflow** para registrar los parámetros, métricas y artefactos de cada modelo, lo que nos permitirá comparar los resultados de manera organizada.

In [9]:
metrics_df = train_baselines_with_mlflow(X, y, transform_target=True, cv_splits=5)

2025/06/20 11:19:20 INFO mlflow.tracking.fluent: Experiment with name 'baseline_models' does not exist. Creating a new experiment.


--- Entrenando modelo: ridge ---
Registrando métricas para ridge: {'val_rmse': np.float64(-4.8926), 'val_mae': np.float64(-2.8653), 'val_medae': np.float64(-1.4413)}
--- Entrenando modelo: lasso ---
Registrando métricas para lasso: {'val_rmse': np.float64(-67.8223), 'val_mae': np.float64(-53.8631), 'val_medae': np.float64(-44.9225)}
--- Entrenando modelo: random_forest ---
Registrando métricas para random_forest: {'val_rmse': np.float64(-1.2707), 'val_mae': np.float64(-0.7971), 'val_medae': np.float64(-0.5296)}
--- Entrenando modelo: hgb ---
Registrando métricas para hgb: {'val_rmse': np.float64(-1.3322), 'val_mae': np.float64(-0.8853), 'val_medae': np.float64(-0.6137)}
--- Entrenando modelo: xgb ---
Registrando métricas para xgb: {'val_rmse': np.float64(-1.4912), 'val_mae': np.float64(-1.0502), 'val_medae': np.float64(-0.7409)}
--- Entrenando modelo: lgbm ---
Registrando métricas para lgbm: {'val_rmse': np.float64(-1.2742), 'val_mae': np.float64(-0.8531), 'val_medae': np.float64(-0.59

In [10]:
metrics_df

,model,rmse,mae,medae
0,ridge,-4.8926,-2.8653,-1.4413
1,lasso,-67.8223,-53.8631,-44.9225
2,random_forest,-1.2707,-0.7971,-0.5296
3,hgb,-1.3322,-0.8853,-0.6137
4,xgb,-1.4912,-1.0502,-0.7409
5,lgbm,-1.2742,-0.8531,-0.5966
6,knn,-5.3983,-3.6924,-2.5979


In [11]:
metrics_df.to_csv("../reports/baseline_metrics.csv", index=False)